# Answering Questions
In this notebook, I hope to answer the following questions, which will be pertinent as we prepare to host our competition.
- What is the distribution of our flagellar motor annotations within their respective tomograms?
- What are the size statistics of tomograms?
- How many slices (axis 0) should we provide to allow competitors to find motors?

In [3]:
from tomograms import all_fm_tomograms

import numpy as np
from matplotlib import pyplot as plt

from visualize_voxels import visualize

In [4]:
all_tomos = all_fm_tomograms()

### What is the distribution of our flagellar motor annotations within their respective tomograms?

In [5]:
def relative_loc(array_shape, point):
    """ Normalizes each component of np array `point` by the components of `array_shape`. """
    return np.array([p / s for (p, s) in zip(point, array_shape)])


def relative_fm_locs(tomogram):
    # Find flagellar motor annotation in tomogram
    annotation = next((a for a in tomogram.annotations 
                        if a.name == "Flagellar Motor"), None)
    if annotation is None:
        return []
    
    rel_locs = [relative_loc(tomogram.get_shape(), point) for point in annotation.points]
    return rel_locs


In [6]:
rel_locs = {} 
for tomo in all_tomos:
    rel_locs[tomo] = relative_fm_locs(tomo)

AttributeError: 'TomogramFile' object has no attribute 'get_shape'

In [7]:
tomos, slices, horiz_positions, vert_positions = ([], [], [], [])
for tomo, locs in rel_locs.items():
    tomos += [tomo for _ in locs]
    slices += [loc[0] for loc in locs]
    horiz_positions += [loc[1] for loc in locs]
    vert_positions  += [loc[2] for loc in locs]

In [8]:
plt.hist(
    slices,
    bins=30,
    orientation='horizontal'
)
plt.xlabel("Number of motors")
plt.ylabel("Slice (normalized to tomogram thickness)")
plt.title("Distribution of flagellar motors by slice")

Text(0.5, 1.0, 'Distribution of flagellar motors by slice')

In [9]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

fig.suptitle("Flagellar motors\nin their respective slice")

ax[0].scatter(
    horiz_positions,
    vert_positions,
    s=10
)
ax[0].set_aspect('equal', adjustable='box')
ax[0].set_xlabel("Horizontal position (normalized)")
ax[0].set_ylabel("Vertical position (normalized)")
ax[0].set_title("Scatter")

ax[1].hist2d(
    horiz_positions,
    vert_positions,
    bins=15
)
ax[1].set_aspect('equal', adjustable='box')
ax[1].set_xlabel("Horizontal position (normalized)")
ax[1].set_ylabel("Vertical position (normalized)")
ax[1].set_title("Heatmap")

fig.tight_layout()

Finding the extreme annotations for Braxton

In [ ]:
sorted_indices = np.argsort(slices)

In [ ]:
# Lowest slices
lowest_motors = [tomos[i] for i in sorted_indices[:3]]

for tomo in lowest_motors:
    print(tomo.filepath)

In [ ]:
# Highest slices
highest_motors = [tomos[i] for i in sorted_indices[-3:]]

for tomo in highest_motors:
    print(tomo.filepath)

In [ ]:
for i in sorted_indices:
    tomo = tomos[i]
    print(tomo.filepath)
    print()

In [ ]:
tomo = lowest_motors[0]
tomo.load()
visualize(tomo.data, marks=tomo.annotations[0].points, title=f"Lowest Motors ({i})")

In [ ]:
tomo = highest_motors[-1]
tomo.load()
visualize(tomo.data, marks=tomo.annotations[0].points, title=f"Highest Motors ({i})")

### What are the size statistics of tomograms?

In [ ]:
shapes = []
for tomo in all_tomos:
    shapes.append(tomo.get_shape())

In [ ]:
n_slices = np.array([shape[0] for shape in shapes])

In [ ]:
print(f"Unique numbers of slices: {np.unique(n_slices)}")

Of the 600 tomograms I can find annotations for, all of them have 300, 500, or 600 slices! 

In [ ]:
plt.hist(n_slices)
plt.xlabel("Slices")
plt.ylabel("Tomograms")
plt.title("Number of slices in tomograms with motor annotations")
plt.show()

In [ ]:
n_horiz = np.array([shape[1] for shape in shapes])
n_vert  = np.array([shape[2] for shape in shapes])

In [ ]:
print(f"Unique horizontal sizes: {np.unique(n_horiz)}")
print(f"Unique vertical sizes:   {np.unique(n_vert) }")


In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib.ticker import ScalarFormatter

# Create a custom colormap where 0 values are white
original_map = plt.cm.viridis
new_cmap = original_map(np.arange(original_map.N))
new_cmap[0, :] = np.array([1, 1, 1, 1])  # Set the color for 0 values to white
custom_cmap = ListedColormap(new_cmap)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, slices in zip(axes, [300, 500, 600]):
    hist = ax.hist2d(
        [h for (s, h) in zip(n_slices, n_horiz) if s == slices],
        [v for (s, v) in zip(n_slices, n_vert)  if s == slices],
        bins=25,
        cmap=custom_cmap
    )
    ax.set_title(f"Shapes of {slices}-slice tomograms (N = {np.sum(n_slices==slices)})")
    cbar = fig.colorbar(hist[3], ax=ax, shrink=0.75)
    cbar.set_label('Tomograms')

    # Force axis to use plain numbers instead of scientific notation
    ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
    ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))

fig.suptitle("~ Shapes of tomograms with motor annotations ~")
fig.tight_layout()

### How many slices (axis 0) should we provide to allow competitors to find motors?